<a href="https://colab.research.google.com/github/Sanjidh090/kaggle_colab_practise/blob/main/Bangla_Sentiment_Analysis_V50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

aiquest_bangla_sentiment_analysis_competition_path = kagglehub.competition_download('aiquest-bangla-sentiment-analysis-competition')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aiquest-bangla-sentiment-analysis-competition/sample_submission.csv
/kaggle/input/aiquest-bangla-sentiment-analysis-competition/train.csv


In [ ]:
train=pd.read_csv("/kaggle/input/aiquest-bangla-sentiment-analysis-competition/train.csv")
sample_sub=pd.read_csv("/kaggle/input/aiquest-bangla-sentiment-analysis-competition/sample_submission.csv")

In [ ]:
import pandas as pd
import re
import unicodedata
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Ensure NLTK resources are downloaded
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Define Bangla stopwords
stop_words = set(stopwords.words('bengali'))

# Step 1: Custom replace rules for company_x, See Translation, Payment Tk etc.
def custom_replace(text):
    # Replace specific payment text
    text = re.sub(r"Payment Tk.*?successful", "আমি আজ কিছু টাকা পরিশোধ করেছি।", text)

    # Replace specific loan sentence
    text = re.sub(
        r"খুব দরকারের টাইমে company_x থেকে সহজে লোন পেলাম\?See Translation",
        "আমি জরুরি সময়ে সহজে লোন পেয়েছিলাম।",
        text
    )

    # Remove |See Translation or See Translation
    text = re.sub(r'\|?See Translation', '', text)

    # Replace company_x, company_y appropriately
    if 'company_x' in text or 'company_y' in text:
        if re.search(r'লোন|সেবা|নক|যোগাযোগ|কল', text):
            text = re.sub(r'company_\w+', 'ওই প্রতিষ্ঠান', text)
        else:
            text = re.sub(r'company_\w+', 'একটি কোম্পানি', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Step 2: Main cleaning function
def full_bangla_text_cleaner(text, remove_english=True, remove_punctuation=True, normalize=True, fix_spacing=True):
    # Apply custom replaces first
    text = custom_replace(text)

    # Lowercase everything
    text = text.lower()

    # Remove English words/numbers
    if remove_english:
        text = re.sub(r'[a-zA-Z0-9_]+', '', text)

    # Remove punctuation
    if remove_punctuation:
        text = re.sub(r'[।॥!?.,;:“”"\'‘’—…()\[\]{}<>@#$%^&*_+=|\\/~`]', '', text)



    # Keep only Bangla characters
    text = re.sub(r'[^\u0980-\u09FF\s]', '', text)

    # Fix spacing issues with common errors
    if fix_spacing:
        text = re.sub(r'(\S+)\s ে', r'\1ে', text)
        text = re.sub(r'(\S+)\s রে', r'\1রে', text)

    # Basic spelling corrections
    common_fixes = {
        "স ুখীন": "সুখীন",
        "সি ম্পল": "সিম্পল",
        "নে ভিগেট": "নেভিগেট",
        "শ ুধু": "শুধু",
        "স ্ছি": "পাচ্ছি",
        "কোম্প ানির": "কোম্পানির",
        "ন েভিগেট": "নেভিগেট",
        "ে নক": "এ নক",
        "বাস্তব ে":"বাস্তবে",
        "সুদ ্ছি" : "সুদ পাচ্ছি",
        "স িম্পল": "সিম্পল",
        "প্রোডাক্ট ের":"প্রোডাক্টের"
    }
    for wrong, right in common_fixes.items():
        text = text.replace(wrong, right)

    # Unicode normalization
    text = unicodedata.normalize('NFKC', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Normalize letters
    if normalize:
        tokens = [re.sub(r'(থ্)(য|য়)', 'থ্য', word) for word in tokens]
        tokens = [re.sub(r'(অ্)(য|য়)', 'অ্য', word) for word in tokens]

    # Stopword filtering
    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)
train["clean_text"] = train["text"].apply(full_bangla_text_cleaner)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

# ✅ Dataset columns: 'clean_text', 'sentiment', 'id'
X_text = train["clean_text"]
y = train["sentiment"]
ids = train["id"]

# ✅ Label Encoding
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# ✅ TF-IDF vectorization
vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 2),
    max_df=0.9,
    min_df=3
)
X_tfidf = vectorizer.fit_transform(X_text)

# ✅ Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weight_dict = dict(zip(np.unique(y_encoded), class_weights))

# ✅ Best hyperparameters (replace with real ones if available)
best_params = {
    "nb_alpha": 0.1,
    "rf_n_estimators": 150,
    "svc_c": 1.0
}

# ✅ Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X_text), dtype=int)

for fold, (train_idx, val_idx) in enumerate(cv.split(X_tfidf, y_encoded)):
    print(f"🔁 Fold {fold+1}")
    X_train_fold, y_train_fold = X_tfidf[train_idx], y_encoded[train_idx]
    X_val_fold, y_val_fold = X_tfidf[val_idx], y_encoded[val_idx]

    # ✅ Stacking Classifier
    model = StackingClassifier(
        estimators=[
            ("nb", MultinomialNB(alpha=best_params["nb_alpha"])),
            ("rf", RandomForestClassifier(n_estimators=best_params["rf_n_estimators"], class_weight=class_weight_dict, random_state=42)),
            ("svc", SVC(C=best_params["svc_c"], probability=True, class_weight=class_weight_dict, random_state=42))
        ],
        final_estimator=LogisticRegression(max_iter=1000, class_weight=class_weight_dict),
        cv=5,
        n_jobs=-1
    )

    model.fit(X_train_fold, y_train_fold)
    preds = model.predict(X_val_fold)
    oof_preds[val_idx] = preds

# ✅ Decode predictions and true labels
true_labels = le.inverse_transform(y_encoded)
oof_labels = le.inverse_transform(oof_preds)

# ✅ Classification Report
print("\n📊 Cross-Validation Classification Report:")
print(classification_report(true_labels, oof_labels))


🔁 Fold 1
🔁 Fold 2
🔁 Fold 3
🔁 Fold 4
🔁 Fold 5

📊 Cross-Validation Classification Report:
              precision    recall  f1-score   support

    negative       0.83      0.83      0.83        65
     neutral       0.88      0.95      0.92        64
    positive       0.91      0.83      0.87        60

    accuracy                           0.87       189
   macro avg       0.87      0.87      0.87       189
weighted avg       0.87      0.87      0.87       189



In [ ]:
#Save submission file
submission = pd.DataFrame({
  "id": ids,
    "sentiment": oof_labels})
submission.to_csv("/kaggle/working/submission.csv", index=False)
print("✅ submission.csv saved.")

✅ submission.csv saved.
